In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
dbutils.widgets.text("target_dataset", "VCM_DWH_PRD") 
dbutils.widgets.text("target_table", "f_staging_inventory")
dbutils.widgets.text("metadata_schema", "udp_wcm_metadata_dev")
dbutils.widgets.text(
    "dependency_table","VCM_STG_ING.sap_vcdmmim01,VCM_STG_INF.sap_vcdmmim02" 
)

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
dbutils.widgets.text("field_calday", "CALDAY")
dbutils.widgets.text("field_id", "HASH_ID")

In [0]:
field_calday = dbutils.widgets.get("field_calday")
field_id = dbutils.widgets.get("field_id")

print(f"field_calday: {field_calday}")
print(f"field_id: {field_id}")

In [0]:
environment = dbutils.widgets.get("environment")
target_dataset = dbutils.widgets.get("target_dataset")
target_table = dbutils.widgets.get("target_table")
metadata_schema = dbutils.widgets.get("metadata_schema")
dependency_table = dbutils.widgets.get("dependency_table")
dependency_table = [x.strip().upper() for x in dependency_table.split(",")]
dependency_table = "'" + "','".join(dependency_table) + "'"

print(f"environment: {environment}")
print(f"target_dataset: {target_dataset}")
print(f"target_table: {target_table}")
print(f"catalog_name: {catalog_name}")
print(f"metadata_schema: {metadata_schema}")
print(f"dependency_table: {dependency_table}")

In [0]:
%run "../common/common_etl_load"

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.f_staging_inventory
(
hash_id BIGINT,
calday DATE,
store_id STRING,
product_id STRING,
closing_stock_quantity FLOAT,
closing_stock_value FLOAT,
receive_stock_quantity FLOAT,
receive_stock_value FLOAT,
issue_stock_quantity FLOAT,
issue_stock_value FLOAT,
receive_consignment_stock_quantity FLOAT,
issue_consignment_stock_quantity FLOAT,
closing_consignment_stock_quantity FLOAT
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
if CALDAY_IN_STR != '':
    spark.sql(f"""
    CREATE OR REPLACE TEMP VIEW temp_f_staging_inventory
    AS
    WITH
    t1 AS (
        SELECT
            calday,
            material,
            plant,
            ROUND(SUM(recvalstck),3) receive_stock_quantity,
            ROUND(SUM(issvalstck),3) issue_stock_quantity,
            ROUND(SUM(CAST(reccnsstck AS FLOAT)),3) receive_consignment_stock_quantity,
            ROUND(SUM(CAST(isscnsstck AS FLOAT)),3) issue_consignment_stock_quantity
        FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_vcdmmim01
        WHERE material IS NOT NULL
        AND calday IN ({CALDAY_IN_STR})
        GROUP BY 1,2,3
    ),
    t2 AS (
        SELECT
            calday,
            material,
            plant,
            CASE
                WHEN SUM(recvs_val) < 0.000001 AND SUM(recvs_val) > 0 THEN 0
                WHEN SUM(recvs_val) > -0.000001 AND SUM(recvs_val) < 0 THEN 0
            ELSE SUM(recvs_val)
            END receive_stock_value,
            CASE
                WHEN SUM(issvs_val) < 0.000001 AND SUM(issvs_val) > 0 THEN 0
                WHEN SUM(issvs_val) > -0.000001 AND SUM(issvs_val) < 0 THEN 0
            ELSE SUM(issvs_val)
            END issue_stock_value
        FROM  {catalog_name}.udp_wcm_silver_sap_bw.sap_vcdmmim02
        WHERE material IS NOT NULL
        AND calday IN ({CALDAY_IN_STR})
        GROUP BY 1,2,3
    )
    SELECT
        farm_fingerprint(CONCAT(
            IFNULL(CAST(IFNULL(t1.calday,t2.calday) AS STRING), ""),
            IFNULL(CAST(IFNULL(t1.plant,t2.plant) AS STRING), ""),
            IFNULL(CAST(IFNULL(t1.material,t2.material) AS STRING), "")
        )) hash_id
        ,IFNULL(t1.calday,t2.calday) calday
        ,IFNULL(t1.plant,t2.plant) store_id
        ,IFNULL(t1.material,t2.material) product_id
        ,ROUND(IFNULL(t1.receive_stock_quantity - t1.issue_stock_quantity,0),3) closing_stock_quantity
        ,IFNULL(t2.receive_stock_value - t2.issue_stock_value,0) closing_stock_value
        ,ROUND(IFNULL(t1.receive_stock_quantity,0),3) receive_stock_quantity
        ,IFNULL(t2.receive_stock_value,0) receive_stock_value
        ,ROUND(IFNULL(t1.issue_stock_quantity,0),3) issue_stock_quantity
        ,IFNULL(t2.issue_stock_value,0) issue_stock_value
        ,ROUND(IFNULL(t1.receive_consignment_stock_quantity,0),3) receive_consignment_stock_quantity
        ,ROUND(IFNULL(t1.issue_consignment_stock_quantity,0),3) issue_consignment_stock_quantity
        ,ROUND(IFNULL(t1.receive_consignment_stock_quantity - t1.issue_consignment_stock_quantity,0),3) closing_consignment_stock_quantity
    FROM t1 FULL JOIN t2 ON
        t1.calday = t2.calday
        AND t1.material = t2.material
        AND t1.plant = t2.plant
    """)

In [0]:
if CALDAY_IN_STR != "":
    # Get the calday list string from temp table
    calday_query = f"""
    SELECT
        CONCAT("'", CONCAT_WS("','", COLLECT_SET(CAST(TO_DATE(calday, 'yyyy-MM-dd') AS STRING))), "'") AS calday_in_str
    FROM temp_{target_table}
    """

    calday_df = spark.sql(calday_query)
    calday_in_str = calday_df.collect()[0][0]
    print(calday_in_str)

    # Only run DELETE if the string is non-empty
    if calday_in_str.strip("'"):  # checks if there's any date inside the quotes
        spark.sql(
            f"""
        DELETE FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_staging_inventory
        WHERE calday IN ({calday_in_str})
        """
        ).display()
    else:
        print("no valid calday's found to delete.")
else:
    print(
        f"temp_{target_table} could't be created as no valid calday's found In common-etl so {catalog_name}.udp_wcm_gold_vcm_dwh.f_staging_inventory can't be deleted using calday's from temp_{target_table}"
    )

In [0]:
if CALDAY_IN_STR != "":
    # Insert rows from temp table into main table
    INSERT_ROW = ", ".join(
        spark.table(f"{catalog_name}.udp_wcm_gold_vcm_dwh.f_staging_inventory").columns
    )
    spark.sql(
        f"""
        INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dwh.f_staging_inventory ({INSERT_ROW})
        SELECT {INSERT_ROW} FROM temp_{target_table}
    """
    ).display()
else:
    print(
        f"temp_{target_table} could't be created as no valid calday's found In common-etl so {catalog_name}.udp_wcm_gold_vcm_dwh.f_staging_inventory can't be inserted using calday's from temp_{target_table}"
    )

In [0]:
spark.sql(f"""DROP VIEW IF EXISTS temp_{target_table}""")

In [0]:
if CALDAY_IN_STR == '':
    dbutils.notebook.exit("No valid calday's found In common-etl to insert data into Table: ETL_LOG_DEPENDENCIES and ETL_DELTA_TABLE and ")

In [0]:
%run "../common/common_etl_update"